## CNN Model

This code has minor differences, mainly related to handling data in a Pandas dataframe.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from sklearn.model_selection import train_test_split

In [2]:
notesInString = [['e string OFF', 'E2', 'F2', 'F#2', 'G2', 'G#2', 'A2', 'A#2', 'B2', 
                  'C3', 'C#3', 'D3', 'D#3', 'E3', 'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3'],
                 ['A string OFF', 'A2', 'A#2', 'B2', 'C3', 'C#3', 'D3', 'D#3', 'E3', 
                  'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3', 'C4', 'C#4', 'D4', 'D#4', 'E4'],
                 ['D string OFF', 'D3', 'D#3', 'E3', 'F3', 'F#3', 'G3', 'G#3', 'A3', 
                  'A#3', 'B3', 'C4', 'C#4', 'D4', 'D#4', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4'],
                 ['G string OFF', 'G3', 'G#3', 'A3', 'A#3', 'B3', 'C4', 'C#4', 'D4', 
                  'D#4', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4', 'A#4', 'B4', 'C5', 'C#5', 'D5'],
                 ['B string OFF', 'B3', 'C4', 'C#4', 'D4', 'D#4', 'E4', 'F4', 'F#4', 
                  'G4', 'G#4', 'A4', 'A#4', 'B4', 'C5', 'C#5', 'D5', 'D#5', 'E5', 'F5', 'F#5'],
                 ['E string OFF', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4', 'A#4', 'B4', 
                  'C5', 'C#5', 'D5', 'D#5', 'E5', 'F5', 'F#5', 'G5', 'G#5', 'A5', 'A#5', 'B5']]

print('Loading dataset...')
ds = pickle.load(open('data/dataset5', 'rb'))
print('Done')


Loading dataset...
Done


In [3]:
train, test = train_test_split(ds, test_size=0.1, random_state=42)

X_train = np.array(train['CQT'].to_list())
X_train = np.reshape(X_train, (len(X_train), 84, 9, 1))

X_test = np.array(test['CQT'].to_list())
X_test = np.reshape(X_test, (len(X_test), 84, 9, 1))

print('Creating one-hot-encoding array')
y_train = []
y_test = []
for c in train.columns[1:]:
    y_train += [tf.keras.utils.to_categorical(train[c], num_classes=21)]
    y_test += [tf.keras.utils.to_categorical(test[c], num_classes=21)]


e_train = y_train[0]
A_train = y_train[1]
D_train = y_train[2]
G_train = y_train[3]
B_train = y_train[4]
E_train = y_train[5]

e_test = y_test[0]
A_test = y_test[1]
D_test = y_test[2]
G_test = y_test[3]
B_test = y_test[4]
E_test = y_test[5]

print(X_train.shape)


Creating one-hot-encoding array
(126730, 84, 9, 1)


In [4]:
batch_size = 32

input_shape = X_train.shape[1:]
print('Input shape: ', input_shape)
# Optimizer
epochs = 30
learning_rate = 0.01 
momentum = 0.8
decay = learning_rate/epochs
sgd = keras.optimizers.SGD(lr = learning_rate, momentum = momentum, decay = decay, nesterov = False)

# Training (Functional Method)
model_in = keras.Input(shape = input_shape)
conv1 = Conv2D(32, kernel_size = (3, 3), activation = 'relu')(model_in)
conv2 = Conv2D(64, kernel_size = (3, 3), activation = 'relu')(conv1)
conv3 = Conv2D(64, kernel_size = (3, 3), activation = 'relu')(conv2)
pool1 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(conv3)
flat = Flatten()(pool1)

# Create fully connected model heads
y1 = Dense(128, activation = 'relu')(flat)
y1 = Dropout(0.5)(y1)
y1 = Dense(126)(y1)
y1 = Dropout(0.2)(y1)

y2 = Dense(128, activation = 'relu')(flat)
y2 = Dropout(0.5)(y2)
y2 = Dense(126)(y2)
y2 = Dropout(0.2)(y2)

y3 = Dense(128, activation = 'relu')(flat)
y3 = Dropout(0.5)(y3)
y3 = Dense(126)(y3)
y3 = Dropout(0.2)(y3)

y4 = Dense(128, activation = 'relu')(flat)
y4 = Dropout(0.5)(y4)
y4 = Dense(126)(y4)
y4 = Dropout(0.2)(y4)

y5 = Dense(128, activation = 'relu')(flat)
y5 = Dropout(0.5)(y5)
y5 = Dense(126)(y5)
y5 = Dropout(0.2)(y5)

y6 = Dense(128, activation = 'relu')(flat)
y6 = Dropout(0.5)(y6)
y6 = Dense(126)(y6)
y6 = Dropout(0.2)(y6)

# Connect heads to final output layer
out1 = Dense(21, activation = 'softmax', name = 'estring')(y1)
out2 = Dense(21, activation = 'softmax', name = 'Astring')(y2)
out3 = Dense(21, activation = 'softmax', name = 'Dstring')(y3)
out4 = Dense(21, activation = 'softmax', name = 'Gstring')(y4)
out5 = Dense(21, activation = 'softmax', name = 'Bstring')(y5)
out6 = Dense(21, activation = 'softmax', name = 'Estring')(y6)

# Create model
model = keras.Model(inputs = model_in, outputs = [out1, out2, out3, out4, out5, out6]) #, out2, out3, out4, out5, out6])
model.compile(optimizer = sgd, loss = ['categorical_crossentropy', 'categorical_crossentropy', 
                                       'categorical_crossentropy', 'categorical_crossentropy', 
                                       'categorical_crossentropy', 'categorical_crossentropy'],
              metrics = ['accuracy'])



Input shape:  (84, 9, 1)


In [5]:

history = model.fit(X_train, [e_train, A_train, D_train, G_train, B_train, E_train],
                    batch_size = batch_size, epochs = epochs, verbose = 1,
                    validation_data = (X_test, [e_test, A_test, D_test, G_test, B_test, E_test]))

model.save('data/model5')


Train on 126730 samples, validate on 14082 samples
Epoch 1/30
126730/126730 [==============================] - 64s 507us/sample - loss: 6.0729 - estring_loss: 0.7111 - Astring_loss: 0.9157 - Dstring_loss: 1.1347 - Gstring_loss: 1.2010 - Bstring_loss: 1.2108 - Estring_loss: 0.8994 - estring_accuracy: 0.7940 - Astring_accuracy: 0.7288 - Dstring_accuracy: 0.6513 - Gstring_accuracy: 0.6215 - Bstring_accuracy: 0.6191 - Estring_accuracy: 0.7246 - val_loss: 3.8352 - val_estring_loss: 0.4533 - val_Astring_loss: 0.6008 - val_Dstring_loss: 0.7170 - val_Gstring_loss: 0.7549 - val_Bstring_loss: 0.7503 - val_Estring_loss: 0.5516 - val_estring_accuracy: 0.8587 - val_Astring_accuracy: 0.8136 - val_Dstring_accuracy: 0.7703 - val_Gstring_accuracy: 0.7608 - val_Bstring_accuracy: 0.7594 - val_Estring_accuracy: 0.8221
Epoch 2/30
126730/126730 [==============================] - 62s 492us/sample - loss: 4.2908 - estring_loss: 0.5010 - Astring_loss: 0.6623 - Dstring_loss: 0.8148 - Gstring_loss: 0.8480 - Bstr

Epoch 12/30
126730/126730 [==============================] - 62s 492us/sample - loss: 2.7878 - estring_loss: 0.3257 - Astring_loss: 0.4437 - Dstring_loss: 0.5428 - Gstring_loss: 0.5565 - Bstring_loss: 0.5511 - Estring_loss: 0.3680 - estring_accuracy: 0.9033 - Astring_accuracy: 0.8661 - Dstring_accuracy: 0.8316 - Gstring_accuracy: 0.8270 - Bstring_accuracy: 0.8271 - Estring_accuracy: 0.8847 - val_loss: 2.3400 - val_estring_loss: 0.2822 - val_Astring_loss: 0.3848 - val_Dstring_loss: 0.4608 - val_Gstring_loss: 0.4743 - val_Bstring_loss: 0.4521 - val_Estring_loss: 0.2810 - val_estring_accuracy: 0.9182 - val_Astring_accuracy: 0.8867 - val_Dstring_accuracy: 0.8626 - val_Gstring_accuracy: 0.8570 - val_Bstring_accuracy: 0.8631 - val_Estring_accuracy: 0.9143
Epoch 13/30
126730/126730 [==============================] - 62s 492us/sample - loss: 2.7332 - estring_loss: 0.3189 - Astring_loss: 0.4376 - Dstring_loss: 0.5339 - Gstring_loss: 0.5436 - Bstring_loss: 0.5403 - Estring_loss: 0.3588 - estring

Epoch 23/30
126730/126730 [==============================] - 62s 492us/sample - loss: 2.4136 - estring_loss: 0.2789 - Astring_loss: 0.3889 - Dstring_loss: 0.4779 - Gstring_loss: 0.4836 - Bstring_loss: 0.4778 - Estring_loss: 0.3064 - estring_accuracy: 0.9158 - Astring_accuracy: 0.8814 - Dstring_accuracy: 0.8495 - Gstring_accuracy: 0.8480 - Bstring_accuracy: 0.8492 - Estring_accuracy: 0.9031 - val_loss: 2.1059 - val_estring_loss: 0.2505 - val_Astring_loss: 0.3482 - val_Dstring_loss: 0.4179 - val_Gstring_loss: 0.4303 - val_Bstring_loss: 0.4116 - val_Estring_loss: 0.2429 - val_estring_accuracy: 0.9287 - val_Astring_accuracy: 0.8980 - val_Dstring_accuracy: 0.8762 - val_Gstring_accuracy: 0.8708 - val_Bstring_accuracy: 0.8771 - val_Estring_accuracy: 0.9256
Epoch 24/30
126730/126730 [==============================] - 62s 491us/sample - loss: 2.3934 - estring_loss: 0.2747 - Astring_loss: 0.3863 - Dstring_loss: 0.4732 - Gstring_loss: 0.4813 - Bstring_loss: 0.4751 - Estring_loss: 0.3027 - estring